In [1]:
import re
from docx import Document
doc=Document('thesis1.docx')
text=" ".join([p.text for p in doc.paragraphs])

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  
    text = re.sub(r"\s+", " ", text)
    return text.strip()
cleaned_text = clean_text(text)
sentences = cleaned_text.split(".") 
words = cleaned_text.split()

preprocessing data

In [4]:
window_size = 5
X = []
y = []
for i in range(window_size, len(words)):
    X.append(words[i-window_size:i])
    y.append(words[i])

tokens[0]

In [5]:
from collections import Counter

import torch



counter = Counter(words)
vocab = ["<pad>", "<unk>"] + list(counter.keys())
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}

vocab_size = len(vocab)
X_idx = [
    [word2idx.get(w, word2idx["<unk>"]) for w in seq]
    for seq in X
]

y_idx = [word2idx.get(w, word2idx["<unk>"]) for w in y]

X = torch.tensor(X_idx)
y = torch.tensor(y_idx)

convert each word to unique number

In [6]:
X=X.cuda()
y=y.cuda()


In [7]:
from gensim.models import FastText
fasttext_model = FastText.load_fasttext_format('cc.fa.300.bin')
embed_dim = 300
embedding_matrix = torch.zeros((vocab_size, embed_dim))
for word, idx in word2idx.items():
  if word in fasttext_model.wv:
    embedding_matrix[idx] = torch.tensor(fasttext_model.wv[word])
    embedding_matrix=embedding_matrix.cuda()
  else:
    embedding_matrix[idx] = torch.randn(embed_dim)
    


C:\Users\USER\AppData\Local\Temp\ipykernel_35136\2279556815.py:2: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext_model = FastText.load_fasttext_format('cc.fa.300.bin')


In [8]:
import torch.nn as nn
import torch.nn as nn
embedding_layer=nn.Embedding.from_pretrained(embedding_matrix,freeze=False)

In [ ]:
import torch
import torch.nn as nn

class NextWordBiLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, vocab_size):
        super().__init__()


        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=True 
        )

        # مقدار embedding_dim رو از nn.Embedding بدست میاریم
        embedding_dim = self.embedding.embedding_dim

    
        self.bilstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )

     
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # (B, T, embedding_dim)
        out, _ = self.bilstm(x)  # (B, T, 2*hidden_dim)
        out = out[:, -1, :]  
        out = self.fc(out)  # (B, vocab_size)
        return out

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NextWordBiLSTM(
    embedding_matrix,
    hidden_dim=128,
    vocab_size=vocab_size
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

X = X.to(device)
y = y.to(device)

for epoch in range(20):
    model.train()
    optimizer.zero_grad()

    outputs = model(X)
    loss = criterion(outputs, y)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

C:\Users\USER\AppData\Local\Temp\ipykernel_35136\3447138470.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embedding_matrix, dtype=torch.float32),


Epoch 1, Loss: 7.497747421264648
Epoch 2, Loss: 7.4865899085998535
Epoch 3, Loss: 7.475114345550537
Epoch 4, Loss: 7.462834358215332
Epoch 5, Loss: 7.449239253997803
Epoch 6, Loss: 7.433797836303711
Epoch 7, Loss: 7.415948867797852
Epoch 8, Loss: 7.395096778869629
Epoch 9, Loss: 7.3705644607543945
Epoch 10, Loss: 7.341572284698486
Epoch 11, Loss: 7.307217121124268
Epoch 12, Loss: 7.266456604003906
Epoch 13, Loss: 7.21812105178833
Epoch 14, Loss: 7.1609649658203125
Epoch 15, Loss: 7.093814373016357
Epoch 16, Loss: 7.015848159790039
Epoch 17, Loss: 6.927062034606934
Epoch 18, Loss: 6.828914642333984
Epoch 19, Loss: 6.724972248077393
Epoch 20, Loss: 6.621257781982422


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
def sample_top_k(output, top_k=5, temperature=0.8):

    logits = output.squeeze() / temperature
    probs = F.softmax(logits, dim=-1).cpu().numpy()
    

    top_indices = probs.argsort()[-top_k:]
    top_probs = probs[top_indices]
    top_probs = top_probs / top_probs.sum()  # نرمال‌سازی
    chosen_idx = np.random.choice(top_indices, p=top_probs)
    
    return chosen_idx

In [ ]:
def generate_text(model, seed_text, next_words=50, seq_length=10,
                  top_k=5, temperature=0.8, device='cuda'):

    model.eval()
    words = seed_text.split()

    for _ in range(next_words):
        # استفاده از آخرین seq_length کلمه
        input_seq = [word2idx.get(w, word2idx["<unk>"]) for w in words[-seq_length:]]
        
        # اگر seed کوتاه بود، با <pad> پر می‌کنیم
        if len(input_seq) < seq_length:
            input_seq = [word2idx["<pad>"]] * (seq_length - len(input_seq)) + input_seq
        
        input_seq = torch.tensor(input_seq).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_seq)
            predicted_idx = sample_top_k(output, top_k=top_k, temperature=temperature)
            predicted_word = idx2word.get(predicted_idx, "<unk>")


        if len(words) > 1 and predicted_word == words[-1]:
            continue  
        
        words.append(predicted_word)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
seed = "تحلیل و شناسایی الگوهای رفتاری"
text = generate_text(model, seed_text=seed, next_words=30, 
                     seq_length=10, top_k=5, temperature=0.8, device=device)
print(text)


None
